# Rc-files HOWTO

The default format for configuration files in lumia is "rc-files". A rc-file contains pairs of `key : value` settings, where `value` can be a single variable or a list (separated by commas), of string, integer, real or boolean format. The keys values can also be set as a composite of the values of other keys.

In python, the settings are stored in an instance of the `lumia.rc` class, which has `setkey` and `get` methods, which enable setting and reading the values of rc-keys.

In [3]:
import lumia
rcf = lumia.rc("rc/gmdd.rc")

The file has the following content:

In [4]:
from IPython import display
display.Pretty(filename="rc/gmdd.rc")

#include common.rc

! settings common to the gmdd inversions

! ################################################################################################
! Paths:
project     : GMD
path.run    : ${LUMIA_SCRATCH}/LUMIA/${project}/${tag}   ! Temporary path run files are stored
path.output : results/${project}/${tag}                  ! Location of the output

! ################################################################################################
! Domain size and time span:
time.start : 2011,1,1
time.end   : 2012,1,1
region     : eurocom05x05

! ################################################################################################
! Fluxes
emissions.categories       : fossil, ocean, fires, biosphere ! flux categories transported

! The fluxes for the category "fossil" will be read from the files "flux_co2.EDGAR_eurocom.YYYY.nc" (with YYYY the year), in the folder ${LUMIA_DATA}/fluxes/eurocom05x05/3h (with ${LUMIA_DATA} an environment variable)
emissions.*.inter


The keys can be simple (e.g. the `project` key here), or composite (e.g. the `path.run` key). The syntax `${keyname}` is used to refer to the value of the key `keyname`, or, if there is no `keyname` key defined, it will look for a `keyname` environment variable as a fallback:

In [24]:
rcf.setkey("tag", "super_preliminary")
print(f"   Raw value of the 'path.output' rc-key: {rcf.keys['path.output']}")
print(f"Parsed value of the 'path.output' rc-key: {rcf.get('path.output')}")

   Raw value of the 'path.output' rc-key: results/${project}/${tag}
Parsed value of the 'path.output' rc-key: results/GMD/super_preliminary


The parsing is done dynamically, so the changes are passed on to the derived keys:

In [23]:
rcf.setkey("tag", "super_cool")
print(f"Parsed value of the 'path.output' rc-key: {rcf.get('path.output')}")

Parsed value of the 'path.output' rc-key: results/GMD/super_cool


The `#include` line can be used to include the content of another rc-file. In the example above, `rcf` contains both the keys of the `rc/gmdd.rc` and of `rc/common.rc` files:

In [26]:
rcf.get("transport.output")

True

The keys can be composed of several subkeys (`subkey1.subkey2.subkey3 : value`). In the rc-file, it is possible to replace one of the subkeys by `*`, to set at once all the keys matching this pattern, as is done with the `emissions.*.optimize` key above:

In [27]:
print(rcf.get("emissions.fossil.optimize", totype=bool))
print(rcf.get("emissions.biosphere.optimize", totype=bool))

False
True


The `lumia.rc` class also includes a `write` methods, which writes all its keys and values to a new rc-file. All the composite keys (`${var}`) and `#include` commands are resolved when writing the file, so it provides a snapshot of the value of the keys at the time when the file is written.
Note that the method will fail if some keys cannot be resolved!